# Classification in PySpark's MLlib Project Solution

### Genre classification
Now it's time to leverage what we learned in the lectures to a REAL classification project! Have you ever wondered what makes us, humans, able to tell apart two songs of different genres? How we do we inherenly know the difference between a pop song and heavy metal? This type of classifcation may seem easy for us, but it's a very difficult challenge for a computer to do. So the question is, could an automatic genre classifcation model be possible? 

For this project we will be classifying songs based on a number of characteristics into a set of 23 electronic genres. This technology could be used by an application like Pandora to recommend songs to users or just create meaningful channels. Super fun!

### Dataset
*beatsdataset.csv*
Each row is an electronic music song. The dataset contains 100 song for each genre among 23 electronic music genres, they were the top (100) songs of their genres on November 2016. The 71 columns are audio features extracted of a two random minutes sample of the file audio. These features have been extracted using pyAudioAnalysis (https://github.com/tyiannak/pyAudioAnalysis).

### Your task
Create an algorithm that classifies songs into the 23 genres provided. Test out several different models and select the highest performing one. Also play around with feature selection methods and finally try to make a recommendation to a user.  

For the feature selection aspect of this project, you may need to get a bit creative if you want to select features from a non-tree algorithm. I did not go over this aspect of PySpark intentionally in the previous lectures to give you chance to get used to researching the PySpark documentation page. Here is the link to the Feature Selectors section of the documentation that just might come in handy: https://spark.apache.org/docs/latest/ml-features.html#feature-selectors

Good luck! Have fun :)


### My approach

I decided to approach this analysis in 4 main steps. 

1. **Create Baseline:** Train and evaluate models on raw data without pre-treating it for outliers, skewness or negative values. This way we can clearly see what effect our transformations have on our analysis. 

2. **Test treatments:** Train and evaluate models on treated data (outliers, skewness and negative values) and compare to baseline (#1).

3. **Feature Selection:** Select the best performing models from the previous two approaches and perform feature selection on it to fine tune it. 

4. **Make a recommendation to a user:** Create a scrip to make a recommendation to a user. I intentionally left this part of the project a bit ambiguous 

### Source
https://www.kaggle.com/caparrini/beatsdataset

First things first... let's create our PySpark instance.

In [1]:
# import findspark
# findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession

# May take awhile locally
spark = SparkSession.builder.appName("ClassificationPS").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

22/10/10 14:10:05 WARN Utils: Your hostname, masoud-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.7.139 instead (on interface wlp2s0)
22/10/10 14:10:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 14:10:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
You are working with 1 core(s)


In [2]:
# Read in dependencies
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.sql.types import StringType, IntegerType
import pyspark.sql.functions as F
from pyspark.sql.functions import when
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import (
    LogisticRegression,
    OneVsRest,
    MultilayerPerceptronClassifier,
    NaiveBayes,
    LinearSVC,
    RandomForestClassifier,
    DecisionTreeClassifier,
    GBTClassifier,
)
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator,
    MulticlassClassificationEvaluator,
)
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

Read in our dataset

In [3]:
path = "Datasets/"
df = spark.read.csv(path + "beatsdataset.csv", inferSchema=True, header=True)

### Check out the dataset

Let's produce a print out of the dataframe so we know what we are working with.

In [4]:
df.limit(6).toPandas()

22/10/10 14:10:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/10/10 14:10:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1-ZCRm, 2-Energym, 3-EnergyEntropym, 4-SpectralCentroidm, 5-SpectralSpreadm, 6-SpectralEntropym, 7-SpectralFluxm, 8-SpectralRolloffm, 9-MFCCs1m, 10-MFCCs2m, 11-MFCCs3m, 12-MFCCs4m, 13-MFCCs5m, 14-MFCCs6m, 15-MFCCs7m, 16-MFCCs8m, 17-MFCCs9m, 18-MFCCs10m, 19-MFCCs11m, 20-MFCCs12m, 21-MFCCs13m, 22-ChromaVector1m, 23-ChromaVector2m, 24-ChromaVector3m, 25-ChromaVector4m, 26-ChromaVector5m, 27-ChromaVector6m, 28-ChromaVector7m, 29-ChromaVector8m, 30-ChromaVector9m, 31-ChromaVector10m, 32-ChromaVector11m, 33-ChromaVector12m, 34-ChromaDeviationm, 35-ZCRstd, 36-Energystd, 37-EnergyEntropystd, 38-SpectralCentroidstd, 39-SpectralSpreadstd, 40-SpectralEntropystd, 41-SpectralFluxstd, 42-SpectralRolloffstd, 43-MFCCs1std, 44-MFCCs

,_c0,1-ZCRm,2-Energym,3-EnergyEntropym,4-SpectralCentroidm,5-SpectralSpreadm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,9-MFCCs1m,...,63-ChromaVector8std,64-ChromaVector9std,65-ChromaVector10std,66-ChromaVector11std,67-ChromaVector12std,68-ChromaDeviationstd,69-BPM,70-BPMconf,71-BPMessentia,class
0,0,0.136440,0.088861,3.201201,0.262825,0.249212,1.114423,0.007003,0.256682,-22.723259,...,0.003431,0.004981,0.010818,0.024001,0.005201,0.015056,133.333333,0.132792,128.0,BigRoom
1,1,0.117039,0.108389,3.194001,0.247657,0.250288,1.065668,0.005387,0.199821,-21.775871,...,0.004461,0.006441,0.007469,0.015499,0.005589,0.019339,120.000000,0.112767,126.0,BigRoom
2,2,0.085308,0.128525,3.123837,0.217205,0.228652,0.789647,0.008247,0.156822,-22.472722,...,0.001529,0.004556,0.007723,0.017482,0.002901,0.022201,133.333333,0.123373,129.0,BigRoom
3,3,0.103050,0.167042,3.150830,0.233593,0.245032,0.967082,0.006571,0.168083,-21.470751,...,0.001591,0.003514,0.009477,0.023162,0.004165,0.015379,133.333333,0.158876,129.0,BigRoom
4,4,0.151730,0.148405,3.194498,0.293730,0.267231,1.353005,0.003872,0.292055,-21.371157,...,0.003945,0.004131,0.011330,0.028188,0.002639,0.019079,133.333333,0.190708,129.0,BigRoom
5,5,0.127047,0.153488,3.221987,0.261693,0.257361,1.090034,0.004943,0.230099,-21.234846,...,0.002986,0.006533,0.010347,0.025008,0.003035,0.019479,133.333333,0.168933,129.0,BigRoom


In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- 1-ZCRm: double (nullable = true)
 |-- 2-Energym: double (nullable = true)
 |-- 3-EnergyEntropym: double (nullable = true)
 |-- 4-SpectralCentroidm: double (nullable = true)
 |-- 5-SpectralSpreadm: double (nullable = true)
 |-- 6-SpectralEntropym: double (nullable = true)
 |-- 7-SpectralFluxm: double (nullable = true)
 |-- 8-SpectralRolloffm: double (nullable = true)
 |-- 9-MFCCs1m: double (nullable = true)
 |-- 10-MFCCs2m: double (nullable = true)
 |-- 11-MFCCs3m: double (nullable = true)
 |-- 12-MFCCs4m: double (nullable = true)
 |-- 13-MFCCs5m: double (nullable = true)
 |-- 14-MFCCs6m: double (nullable = true)
 |-- 15-MFCCs7m: double (nullable = true)
 |-- 16-MFCCs8m: double (nullable = true)
 |-- 17-MFCCs9m: double (nullable = true)
 |-- 18-MFCCs10m: double (nullable = true)
 |-- 19-MFCCs11m: double (nullable = true)
 |-- 20-MFCCs12m: double (nullable = true)
 |-- 21-MFCCs13m: double (nullable = true)
 |-- 22-ChromaVector1m: double (null

### How many classes do we have and are they balanced?

Just making sure :) 

We have a perfectly balanced dataset, however we only have 100 examples from each class which may make training a decent model challenging. But let's see what we can do!

*Note: This almost never happens in real life :)*

In [6]:
df.groupBy("class").count().show(100)

+--------------------+-----+
|               class|count|
+--------------------+-----+
|           PsyTrance|  100|
|           HardDance|  100|
|              Breaks|  100|
|  HardcoreHardTechno|  100|
|   IndieDanceNuDisco|  100|
|              Trance|  100|
|           DeepHouse|  100|
|ElectronicaDowntempo|  100|
|           ReggaeDub|  100|
|             Minimal|  100|
|         DrumAndBass|  100|
|             Dubstep|  100|
|             BigRoom|  100|
|              Techno|  100|
|               House|  100|
|         FutureHouse|  100|
|        ElectroHouse|  100|
|           GlitchHop|  100|
|           TechHouse|  100|
|              HipHop|  100|
|           FunkRAndB|  100|
|               Dance|  100|
|    ProgressiveHouse|  100|
+--------------------+-----+



## Set up our Data Formatting Function 

Remember that MLlib requires all input columns of your dataframe to be vectorized and our dependent variable needs to be zero indexed. We can do that using our handy dandy function we developed in the lecture. Feel free to make this your own!

For example, with this go, I added a print statement to show the new label values for each class. 

In [7]:
# Data Prep function
def MLClassifierDFPrep(
    df, input_columns, dependent_var, treat_outliers=True, treat_neg_values=True
):

    # change label (class variable) to string type to prep for reindexing
    # Pyspark is expecting a zero indexed integer for the label column.
    # Just incase our data is not in that format... we will treat it by using the StringIndexer built in method
    renamed = df.withColumn(
        "label_str", df[dependent_var].cast(StringType())
    )  # Rename and change to string type
    indexer = StringIndexer(
        inputCol="label_str", outputCol="label"
    )  # Pyspark is expecting the this naming convention
    indexed = indexer.fit(renamed).transform(renamed)
    print(indexed.groupBy("class", "label").count().show(100))

    # Convert all string type data in the input column list to numeric
    # Otherwise the Algorithm will not be able to process it
    numeric_inputs = []
    string_inputs = []
    for column in input_columns:
        if indexed.schema[column].dataType == StringType():
            indexer = StringIndexer(inputCol=column, outputCol=column + "_num")
            indexed = indexer.fit(indexed).transform(indexed)
            new_col_name = column + "_num"
            string_inputs.append(new_col_name)
        else:
            numeric_inputs.append(column)

    if treat_outliers == True:
        print("We are correcting for non normality now!")
        # empty dictionary d
        d = {}
        # Create a dictionary of quantiles
        for col in numeric_inputs:
            d[col] = indexed.approxQuantile(
                col, [0.01, 0.99], 0.25
            )  # if you want to make it go faster increase the last number
        # Now fill in the values
        for col in numeric_inputs:
            skew = indexed.agg(F.skewness(indexed[col])).collect()  # check for skewness
            skew = skew[0][0]
            # This function will floor, cap and then log+1 (just in case there are 0 values)
            if skew > 1:
                indexed = indexed.withColumn(
                    col,
                    F.log(
                        when(df[col] < d[col][0], d[col][0])
                        .when(indexed[col] > d[col][1], d[col][1])
                        .otherwise(indexed[col])
                        + 1
                    ).alias(col),
                )
                print(
                    col + " has been treated for positive (right) skewness. (skew =)",
                    skew,
                    ")",
                )
            elif skew < -1:
                indexed = indexed.withColumn(
                    col,
                    exp(
                        when(df[col] < d[col][0], d[col][0])
                        .when(indexed[col] > d[col][1], d[col][1])
                        .otherwise(indexed[col])
                    ).alias(col),
                )
                print(
                    col + " has been treated for negative (left) skewness. (skew =",
                    skew,
                    ")",
                )

    # Produce a warning if there are negative values in the dataframe that Naive Bayes cannot be used.
    # Note: we only need to check the numeric input values since anything that is indexed won't have negative values
    minimums = df.select(
        [F.min(c).alias(c) for c in df.columns if c in numeric_inputs]
    )  # Calculate the mins for all columns in the df
    min_array = minimums.select(
        F.array(numeric_inputs).alias("mins")
    )  # Create an array for all mins and select only the input cols
    df_minimum = min_array.select(
        F.array_min(min_array.mins)
    ).collect()  # Collect golobal min as Python object
    df_minimum = df_minimum[0][0]  # Slice to get the number itself

    features_list = numeric_inputs + string_inputs
    assembler = VectorAssembler(inputCols=features_list, outputCol="features")
    output = assembler.transform(indexed).select("features", "label")

    #     final_data = output.select('features','label') #drop everything else

    # Now check for negative values and ask user if they want to correct that?
    if df_minimum < 0:
        print(" ")
        print(
            "WARNING: The Naive Bayes Classifier will not be able to process your dataframe as it contains negative values"
        )
        print(" ")

    if treat_neg_values == True:
        print(
            "You have opted to correct that by rescaling all your features to a range of 0 to 1"
        )
        print(" ")
        print("We are rescaling you dataframe....")
        scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

        # Compute summary statistics and generate MinMaxScalerModel
        scalerModel = scaler.fit(output)

        # rescale each feature to range [min, max].
        scaled_data = scalerModel.transform(output)
        final_data = scaled_data.select(
            "label", "scaledFeatures"
        )  # added class to the selection
        final_data = final_data.withColumnRenamed("scaledFeatures", "features")
        print("Done!")

    else:
        print(
            "You have opted not to correct that therefore you will not be able to use to Naive Bayes classifier"
        )
        print("We will return the dataframe unscaled.")
        final_data = output

    return final_data

## Set up our Training and Evaluation Function

Let's use our handy dandy function to train and test all our classifiers we have available to us! I made two modifications here to show you just how flexible this framework is. 

1. You'll see in the area where we print our coefficients and feature importances values, that I created a new output format that joins the values with their corresponding feature names so we can clearly see which value belongs to which feature. I did this by zipping the lists together and creating a Spark dataframe to allow the output to be viewed in column row format as opposed to a messy list. 
2. The second change I added was a parameter for the folds so that I could specify the folds more easily with each iteration. 

Let's check it out!

For more info on available hyper parameters visit: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.classification

In [8]:
def ClassTrainEval(classifier, features, classes, folds, train, test):
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__

        return Mtype

    Mtype = FindMtype(classifier)

    def IntanceFitModel(Mtype, classifier, classes, features, folds, train):

        if Mtype == "OneVsRest":
            # instantiate the base classifier.
            lr = LogisticRegression()
            # instantiate the One Vs Rest Classifier.
            OVRclassifier = OneVsRest(classifier=lr)
            #             fitModel = OVRclassifier.fit(train)
            # Add parameters of your choice here:
            paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]).build()
            # Cross Validator requires the following parameters:
            crossval = CrossValidator(
                estimator=OVRclassifier,
                estimatorParamMaps=paramGrid,
                evaluator=MulticlassClassificationEvaluator(),
                numFolds=folds,
            )  # 3 is best practice
            # Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
        if Mtype == "MultilayerPerceptronClassifier":
            # specify layers for the neural network:
            # input layer of size features, two intermediate of features+1 and same size as features
            # and output of size number of classes
            # Note: crossvalidator cannot be used here
            features_count = len(features[0][0])
            layers = [features_count, features_count + 1, features_count, classes]
            MPC_classifier = MultilayerPerceptronClassifier(
                maxIter=100, layers=layers, blockSize=128, seed=1234
            )
            fitModel = MPC_classifier.fit(train)
            return fitModel
        if (
            Mtype in ("LinearSVC", "GBTClassifier") and classes != 2
        ):  # These classifiers currently only accept binary classification
            print(
                Mtype,
                " could not be used because PySpark currently only accepts binary classification data for this algorithm",
            )
            return
        if Mtype in (
            "LogisticRegression",
            "NaiveBayes",
            "RandomForestClassifier",
            "GBTClassifier",
            "LinearSVC",
            "DecisionTreeClassifier",
        ):

            # Add parameters of your choice here:
            if Mtype in ("LogisticRegression"):
                paramGrid = (
                    ParamGridBuilder()  #                              .addGrid(classifier.regParam, [0.1, 0.01]) \
                    .addGrid(classifier.maxIter, [10, 15, 20])
                    .build()
                )

            # Add parameters of your choice here:
            if Mtype in ("NaiveBayes"):
                paramGrid = (
                    ParamGridBuilder()
                    .addGrid(classifier.smoothing, [0.0, 0.2, 0.4, 0.6])
                    .build()
                )

            # Add parameters of your choice here:
            if Mtype in ("RandomForestClassifier"):
                paramGrid = (
                    ParamGridBuilder().addGrid(classifier.maxDepth, [2, 5, 10])
                    #                                .addGrid(classifier.maxBins, [5, 10, 20])
                    #                                .addGrid(classifier.numTrees, [5, 20, 50])
                    .build()
                )

            # Add parameters of your choice here:
            if Mtype in ("GBTClassifier"):
                paramGrid = (
                    ParamGridBuilder()  #                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
                    #                              .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                    .addGrid(classifier.maxIter, [10, 15, 50, 100]).build()
                )

            # Add parameters of your choice here:
            if Mtype in ("LinearSVC"):
                paramGrid = (
                    ParamGridBuilder()
                    .addGrid(classifier.maxIter, [10, 15])
                    .addGrid(classifier.regParam, [0.1, 0.01])
                    .build()
                )

            # Add parameters of your choice here:
            if Mtype in ("DecisionTreeClassifier"):
                paramGrid = (
                    ParamGridBuilder()  #                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
                    .addGrid(classifier.maxBins, [10, 20, 40, 80, 100])
                    .build()
                )

            # Cross Validator requires all of the following parameters:
            crossval = CrossValidator(
                estimator=classifier,
                estimatorParamMaps=paramGrid,
                evaluator=MulticlassClassificationEvaluator(),
                numFolds=folds,
            )  # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel

    fitModel = IntanceFitModel(Mtype, classifier, classes, features, folds, train)

    # Print feature selection metrics
    if fitModel is not None:

        if Mtype in ("OneVsRest"):
            # Get Best Model
            BestModel = fitModel.bestModel
            global OVR_BestModel
            OVR_BestModel = BestModel
            print(" ")
            print("\033[1m" + Mtype + "\033[0m")
            # Extract list of binary models
            models = BestModel.models
            for model in models:
                print("\033[1m" + "Intercept: " + "\033[0m", model.intercept)
                print("\033[1m" + "Top 20 Coefficients:" + "\033[0m")
                coeff_array = model.coefficients.toArray()
                coeff_scores = []
                for x in coeff_array:
                    coeff_scores.append(float(x))
                # Then zip with input_columns list and create a df
                result = spark.createDataFrame(
                    zip(input_columns, coeff_scores), schema=["feature", "coeff"]
                )
                print(result.orderBy(result["coeff"].desc()).show(truncate=False))

        if Mtype == "MultilayerPerceptronClassifier":
            print("")
            print("\033[1m" + Mtype + "\033[0m")
            print("\033[1m" + "Model Weights: " + "\033[0m", fitModel.weights.size)
            print("")
            global MLPC_Model
            MLPC_BestModel = fitModel

        if Mtype in (
            "DecisionTreeClassifier",
            "GBTClassifier",
            "RandomForestClassifier",
        ):
            # FEATURE IMPORTANCES
            # Estimate of the importance of each feature.
            # Each feature’s importance is the average of its importance across all trees
            # in the ensemble The importance vector is normalized to sum to 1.
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print("\033[1m" + Mtype, " Top 20 Feature Importances" + "\033[0m")
            print("(Scores add up to 1)")
            print("Lowest score is the least important")
            print(" ")
            featureImportances = BestModel.featureImportances.toArray()
            # Convert from numpy array to list
            imp_scores = []
            for x in featureImportances:
                imp_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(
                zip(input_columns, imp_scores), schema=["feature", "score"]
            )
            print(result.orderBy(result["score"].desc()).show(truncate=False))

            # Save the feature importance values and the models
            if Mtype in ("DecisionTreeClassifier"):
                global DT_featureimportances
                DT_featureimportances = BestModel.featureImportances.toArray()
                global DT_BestModel
                DT_BestModel = BestModel
            if Mtype in ("GBTClassifier"):
                global GBT_featureimportances
                GBT_featureimportances = BestModel.featureImportances.toArray()
                global GBT_BestModel
                GBT_BestModel = BestModel
            if Mtype in ("RandomForestClassifier"):
                global RF_featureimportances
                RF_featureimportances = BestModel.featureImportances.toArray()
                global RF_BestModel
                RF_BestModel = BestModel

        # Print the coefficients
        if Mtype in ("LogisticRegression"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print("\033[1m" + Mtype + "\033[0m")
            print("Intercept: " + str(BestModel.interceptVector))
            print("\033[1m" + " Top 20 Coefficients" + "\033[0m")
            print("You should compares these relative to eachother")
            # Convert from numpy array to list
            coeff_array = BestModel.coefficientMatrix.toArray()
            coeff_scores = []
            for x in coeff_array[0]:
                coeff_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(
                zip(input_columns, coeff_scores), schema=["feature", "coeff"]
            )
            print(result.orderBy(result["coeff"].desc()).show(truncate=False))
            # Save the coefficient values and the models
            global LR_coefficients
            LR_coefficients = BestModel.coefficientMatrix.toArray()
            global LR_BestModel
            LR_BestModel = BestModel

        # Print the Coefficients
        if Mtype in ("LinearSVC"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print("\033[1m" + Mtype + "\033[0m")
            print("Intercept: " + str(BestModel.intercept))
            print("\033[1m" + "Top 20 Coefficients" + "\033[0m")
            print("You should compares these relative to eachother")
            #             print("Coefficients: \n" + str(BestModel.coefficients))
            coeff_array = BestModel.coefficients.toArray()
            coeff_scores = []
            for x in coeff_array:
                coeff_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(
                zip(input_columns, coeff_scores), schema=["feature", "coeff"]
            )
            print(result.orderBy(result["coeff"].desc()).show(truncate=False))
            # Save the coefficient values and the models
            global LSVC_coefficients
            LSVC_coefficients = BestModel.coefficients.toArray()
            global LSVC_BestModel
            LSVC_BestModel = BestModel

    # Set the column names to match the external results dataframe that we will join with later:
    columns = ["Classifier", "Result"]

    if Mtype in ("LinearSVC", "GBTClassifier") and classes != 2:
        Mtype = [Mtype]  # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype, score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(
            metricName="accuracy"
        )  # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions)) * 100
        Mtype = [Mtype]  # make this a string
        score = [str(accuracy)]  # make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype, score), schema=columns)
        result = result.withColumn("Result", result.Result.substr(0, 5))

    return result
    # Also returns the fit model important scores or p values

## Testing Time! 

Now we can use these functions above to train and evaluate our models in different ways. I'll show you my approach here but everyone has their own style and this process will likley be much more involved in a real use case, but I just want to show you the art of the possible here quickly. 

In [9]:
# Set up independ and dependent vars
input_columns = df.columns
# keep only relevant columns: everything but the first and last cols
input_columns = input_columns[1:-1]
dependent_var = "class"

# Learn how many classes there are in order to specify evaluation type based on binary or multi and turn the df into an object
class_count = df.select(F.countDistinct("class")).collect()
classes = class_count[0][0]

### Test 1: Without outlier treatment, skew or negative value treatment

This first go will act as our baseline so we can understand how our treatments affected our analysis. So I will opt out of the option for outlier treatment, skew treatment or negative value treatment which means we cannot use the naive bayes classifier. 

In [10]:
# Call on data prep, train and evaluate functions
test1_data = MLClassifierDFPrep(
    df, input_columns, dependent_var, treat_outliers=False, treat_neg_values=False
)
test1_data.limit(5).toPandas()

# Comment out Naive Bayes if your data still contains negative values
classifiers = [
    LogisticRegression(),
    OneVsRest(),
    LinearSVC(),
    RandomForestClassifier(),
    GBTClassifier(),
    DecisionTreeClassifier(),
    MultilayerPerceptronClassifier(),
]

train, test = test1_data.randomSplit([0.7, 0.3])
features = test1_data.select(["features"]).collect()
folds = 2  # because we have limited data

# set up your results table
columns = ["Classifier", "Result"]
vals = [("Place Holder", "N/A")]
results = spark.createDataFrame(vals, columns)

for classifier in classifiers:
    new_result = ClassTrainEval(classifier, features, classes, folds, train, test)
    results = results.union(new_result)
results = results.where("Classifier!='Place Holder'")
print("!!!!!Final Results!!!!!!!!")
results.show(100, False)

+--------------------+-----+-----+
|               class|label|count|
+--------------------+-----+-----+
|           ReggaeDub| 19.0|  100|
|           FunkRAndB|  8.0|  100|
|              Trance| 22.0|  100|
|              Techno| 21.0|  100|
|             Dubstep|  5.0|  100|
|   IndieDanceNuDisco| 15.0|  100|
|ElectronicaDowntempo|  7.0|  100|
|               House| 14.0|  100|
|  HardcoreHardTechno| 12.0|  100|
|              HipHop| 13.0|  100|
|           GlitchHop| 10.0|  100|
|         DrumAndBass|  4.0|  100|
|           PsyTrance| 18.0|  100|
|    ProgressiveHouse| 17.0|  100|
|        ElectroHouse|  6.0|  100|
|           HardDance| 11.0|  100|
|             Minimal| 16.0|  100|
|           TechHouse| 20.0|  100|
|           DeepHouse|  3.0|  100|
|             BigRoom|  0.0|  100|
|         FutureHouse|  9.0|  100|
|               Dance|  2.0|  100|
|              Breaks|  1.0|  100|
+--------------------+-----+-----+

None
 
 
You have opted not to correct that therefore 

 
OneVsRest
Intercept:  -2.383911077336528
Top 20 Coefficients:
+--------------------+------------------+
|feature             |coeff             |
+--------------------+------------------+
|22-ChromaVector1m   |36.20928533345211 |
|63-ChromaVector8std |30.137658408648086|
|27-ChromaVector6m   |29.669419405815542|
|30-ChromaVector9m   |22.509754704213215|
|5-SpectralSpreadm   |12.046726399753203|
|33-ChromaVector12m  |9.31980381615339  |
|26-ChromaVector5m   |8.676506309912744 |
|52-MFCCs10std       |8.65415473516657  |
|54-MFCCs12std       |8.548064768907103 |
|36-Energystd        |6.879070463813826 |
|2-Energym           |6.6806866804076925|
|53-MFCCs11std       |6.283001635659378 |
|65-ChromaVector10std|6.010043197214642 |
|49-MFCCs7std        |2.8089015077757256|
|24-ChromaVector3m   |2.772215877698493 |
|3-EnergyEntropym    |2.4453466736961795|
|47-MFCCs5std        |2.0629765124968826|
|58-ChromaVector3std |1.8883523649114058|
|50-MFCCs8std        |1.6778139740987168|
|18-MFCCs10m

+---------------------+------------------+
|feature              |coeff             |
+---------------------+------------------+
|64-ChromaVector9std  |88.76715352305038 |
|63-ChromaVector8std  |87.65097763189074 |
|68-ChromaDeviationstd|47.073275428334995|
|65-ChromaVector10std |36.85189531435714 |
|67-ChromaVector12std |34.15366795943196 |
|22-ChromaVector1m    |33.257353792244054|
|57-ChromaVector2std  |27.817608982673576|
|60-ChromaVector5std  |20.995374067283716|
|26-ChromaVector5m    |18.39327316141913 |
|39-SpectralSpreadstd |17.67348063484858 |
|62-ChromaVector7std  |17.05106029795166 |
|31-ChromaVector10m   |14.63762948801698 |
|32-ChromaVector11m   |11.301145263643935|
|56-ChromaVector1std  |9.714063209678834 |
|50-MFCCs8std         |9.333105524624516 |
|61-ChromaVector6std  |7.010592287714693 |
|55-MFCCs13std        |6.303528973431229 |
|48-MFCCs6std         |5.422982030776995 |
|58-ChromaVector3std  |4.191928351833847 |
|36-Energystd         |3.717229483226414 |
+----------

+----------------------+------------------+
|feature               |coeff             |
+----------------------+------------------+
|29-ChromaVector8m     |190.21559762171117|
|7-SpectralFluxm       |65.05167313711075 |
|41-SpectralFluxstd    |43.80524234825267 |
|30-ChromaVector9m     |36.138409274106884|
|64-ChromaVector9std   |35.77262565554232 |
|33-ChromaVector12m    |33.25418233430853 |
|39-SpectralSpreadstd  |30.46058409885565 |
|25-ChromaVector4m     |29.162253219702734|
|22-ChromaVector1m     |28.9874875319473  |
|27-ChromaVector6m     |27.688717199759164|
|38-SpectralCentroidstd|19.59671826633898 |
|59-ChromaVector4std   |19.17236625595907 |
|34-ChromaDeviationm   |13.744312038272351|
|56-ChromaVector1std   |11.90496299740165 |
|23-ChromaVector2m     |9.684970892487517 |
|32-ChromaVector11m    |9.484135132024269 |
|28-ChromaVector7m     |3.6064888267480844|
|58-ChromaVector3std   |3.405085429397761 |
|47-MFCCs5std          |3.4041816699168557|
|48-MFCCs6std          |3.235789

LinearSVC  could not be used because PySpark currently only accepts binary classification data for this algorithm
22/10/10 14:14:12 WARN DAGScheduler: Broadcasting large task binary with size 1314.1 KiB
22/10/10 14:14:12 WARN DAGScheduler: Broadcasting large task binary with size 1792.3 KiB
22/10/10 14:14:13 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/10/10 14:14:13 WARN DAGScheduler: Broadcasting large task binary with size 1851.5 KiB
22/10/10 14:14:17 WARN DAGScheduler: Broadcasting large task binary with size 1358.6 KiB
22/10/10 14:14:17 WARN DAGScheduler: Broadcasting large task binary with size 1862.2 KiB
22/10/10 14:14:18 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
22/10/10 14:14:18 WARN DAGScheduler: Broadcasting large task binary with size 1925.3 KiB
22/10/10 14:14:20 WARN DAGScheduler: Broadcasting large task binary with size 1546.3 KiB
22/10/10 14:14:21 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


22/10/10 14:14:22 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


 
RandomForestClassifier  Top 20 Feature Importances
(Scores add up to 1)
Lowest score is the least important
 
+----------------------+--------------------+
|feature               |score               |
+----------------------+--------------------+
|71-BPMessentia        |0.08897862759911365 |
|69-BPM                |0.04408726338876078 |
|44-MFCCs2std          |0.027311978577962238|
|1-ZCRm                |0.02664795138420216 |
|70-BPMconf            |0.023519061321211475|
|38-SpectralCentroidstd|0.022678202256740672|
|2-Energym             |0.02235444804535833 |
|3-EnergyEntropym      |0.020913624267681966|
|52-MFCCs10std         |0.02029471714190821 |
|9-MFCCs1m             |0.019302168944088212|
|39-SpectralSpreadstd  |0.019283611408564882|
|53-MFCCs11std         |0.0189166890080225  |
|55-MFCCs13std         |0.017642317971534444|
|50-MFCCs8std          |0.01762804826085447 |
|24-ChromaVector3m     |0.016087469128535085|
|4-SpectralCentroidm   |0.0159867237896914  |
|43-MFCCs1std 

### Test 2: Test treatments

Train and evaluate models on treated data (outliers, skewness and negative values) and compare to baseline (#1).

In [11]:
# Call on data prep, train and evaluate functions
test2_data = MLClassifierDFPrep(
    df, input_columns, dependent_var, treat_outliers=True, treat_neg_values=True
)
test2_data.limit(5).toPandas()

# Comment out Naive Bayes if your data still contains negative values
classifiers = [
    LogisticRegression(),
    OneVsRest(),
    LinearSVC(),
    NaiveBayes(),
    RandomForestClassifier(),
    GBTClassifier(),
    DecisionTreeClassifier(),
    MultilayerPerceptronClassifier(),
]

train, test = test2_data.randomSplit([0.7, 0.3])
features = test2_data.select(["features"]).collect()
folds = 2

# set up your results table
columns = ["Classifier", "Result"]
vals = [("Place Holder", "N/A")]
results = spark.createDataFrame(vals, columns)

for classifier in classifiers:
    new_result = ClassTrainEval(classifier, features, classes, folds, train, test)
    results = results.union(new_result)
results = results.where("Classifier!='Place Holder'")
print("!!!!!Final Results!!!!!!!!")
results.show(100, False)

+--------------------+-----+-----+
|               class|label|count|
+--------------------+-----+-----+
|           ReggaeDub| 19.0|  100|
|           FunkRAndB|  8.0|  100|
|              Trance| 22.0|  100|
|              Techno| 21.0|  100|
|             Dubstep|  5.0|  100|
|   IndieDanceNuDisco| 15.0|  100|
|ElectronicaDowntempo|  7.0|  100|
|               House| 14.0|  100|
|  HardcoreHardTechno| 12.0|  100|
|              HipHop| 13.0|  100|
|           GlitchHop| 10.0|  100|
|         DrumAndBass|  4.0|  100|
|           PsyTrance| 18.0|  100|
|    ProgressiveHouse| 17.0|  100|
|        ElectroHouse|  6.0|  100|
|           HardDance| 11.0|  100|
|             Minimal| 16.0|  100|
|           TechHouse| 20.0|  100|
|           DeepHouse|  3.0|  100|
|             BigRoom|  0.0|  100|
|         FutureHouse|  9.0|  100|
|               Dance|  2.0|  100|
|              Breaks|  1.0|  100|
+--------------------+-----+-----+

None
We are correcting for non normality now!
7-Spectr

 
OneVsRest
Intercept:  -4.944771186228045
Top 20 Coefficients:
+-------------------+-------------------+
|feature            |coeff              |
+-------------------+-------------------+
|9-MFCCs1m          |4.0369364695874275 |
|71-BPMessentia     |2.2971063072963056 |
|5-SpectralSpreadm  |1.5922944574584637 |
|3-EnergyEntropym   |1.4644438106279072 |
|24-ChromaVector3m  |1.4487273691610163 |
|19-MFCCs11m        |1.4283727666863266 |
|22-ChromaVector1m  |1.3035952906024157 |
|54-MFCCs12std      |1.1183916431980465 |
|52-MFCCs10std      |1.1154152681053153 |
|12-MFCCs4m         |1.0281781755271635 |
|53-MFCCs11std      |1.02652809801967   |
|11-MFCCs3m         |0.8690260994743183 |
|2-Energym          |0.8510788802759686 |
|46-MFCCs4std       |0.7647485205089497 |
|43-MFCCs1std       |0.6933683750540659 |
|64-ChromaVector9std|0.6602884378878567 |
|27-ChromaVector6m  |0.5685863503965541 |
|18-MFCCs10m        |0.561489920391236  |
|62-ChromaVector7std|0.5468188485747335 |
|50-MFCCs8st

+----------------------+------------------+
|feature               |coeff             |
+----------------------+------------------+
|9-MFCCs1m             |4.661764579219151 |
|3-EnergyEntropym      |4.079260305895214 |
|71-BPMessentia        |2.5095567695474466|
|63-ChromaVector8std   |2.1068491633115625|
|62-ChromaVector7std   |1.9857386169100812|
|36-Energystd          |1.8624448763527324|
|43-MFCCs1std          |1.834820457893099 |
|38-SpectralCentroidstd|1.5845603451940233|
|58-ChromaVector3std   |1.4127619545518328|
|41-SpectralFluxstd    |1.3995286799817679|
|17-MFCCs9m            |1.3914731976070263|
|57-ChromaVector2std   |1.3587868095905424|
|46-MFCCs4std          |1.2698712168489557|
|60-ChromaVector5std   |1.1594556202541397|
|67-ChromaVector12std  |1.080547838027102 |
|42-SpectralRolloffstd |1.0406496047927773|
|40-SpectralEntropystd |1.040432784637493 |
|44-MFCCs2std          |1.0174031189060453|
|55-MFCCs13std         |0.9676426462663893|
|2-Energym             |0.932070

+----------------------+------------------+
|feature               |coeff             |
+----------------------+------------------+
|10-MFCCs2m            |2.5281376744995754|
|61-ChromaVector6std   |2.07506107684598  |
|21-MFCCs13m           |2.043585838076033 |
|55-MFCCs13std         |1.7581678226790218|
|26-ChromaVector5m     |1.7148680866602615|
|32-ChromaVector11m    |1.663894809875785 |
|12-MFCCs4m            |1.647498482926067 |
|36-Energystd          |1.622111055816263 |
|38-SpectralCentroidstd|1.6120010654142758|
|20-MFCCs12m           |1.5661064545246517|
|58-ChromaVector3std   |1.2769216334327773|
|70-BPMconf            |1.2547237506972306|
|27-ChromaVector6m     |1.2272687611157977|
|31-ChromaVector10m    |1.184456092236059 |
|30-ChromaVector9m     |1.0208693275463208|
|66-ChromaVector11std  |0.9696730433045065|
|63-ChromaVector8std   |0.9455541722505495|
|51-MFCCs9std          |0.9263343621906617|
|42-SpectralRolloffstd |0.8801761770446627|
|17-MFCCs9m            |0.876780

LinearSVC  could not be used because PySpark currently only accepts binary classification data for this algorithm
22/10/10 14:18:36 WARN DAGScheduler: Broadcasting large task binary with size 1000.0 KiB
22/10/10 14:18:36 WARN DAGScheduler: Broadcasting large task binary with size 1440.1 KiB
22/10/10 14:18:37 WARN DAGScheduler: Broadcasting large task binary with size 1951.2 KiB
22/10/10 14:18:38 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/10/10 14:18:38 WARN DAGScheduler: Broadcasting large task binary with size 1988.6 KiB
22/10/10 14:18:42 WARN DAGScheduler: Broadcasting large task binary with size 1430.9 KiB
22/10/10 14:18:42 WARN DAGScheduler: Broadcasting large task binary with size 1953.5 KiB
22/10/10 14:18:43 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
22/10/10 14:18:43 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
22/10/10 14:18:45 WARN DAGScheduler: Broadcasting large task binary with size 1078.5 KiB
22/10

22/10/10 14:18:48 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


 
RandomForestClassifier  Top 20 Feature Importances
(Scores add up to 1)
Lowest score is the least important
 
+----------------------+--------------------+
|feature               |score               |
+----------------------+--------------------+
|71-BPMessentia        |0.09566278888808244 |
|69-BPM                |0.044277227191660364|
|70-BPMconf            |0.028500117484445933|
|2-Energym             |0.025520839607873737|
|3-EnergyEntropym      |0.02477506153282873 |
|1-ZCRm                |0.02466595415478726 |
|44-MFCCs2std          |0.021905912564285333|
|39-SpectralSpreadstd  |0.021832078203876053|
|38-SpectralCentroidstd|0.019683145581736807|
|54-MFCCs12std         |0.018787629968780698|
|43-MFCCs1std          |0.018004816618415173|
|10-MFCCs2m            |0.017761338213560227|
|52-MFCCs10std         |0.0174125614112311  |
|4-SpectralCentroidm   |0.017034871605926167|
|53-MFCCs11std         |0.016251961691378124|
|41-SpectralFluxstd    |0.016138205903069722|
|55-MFCCs13std

## Test 3: Feature Selection

Looks like all our of our models saw a little bit of improvement from the transformations we applied. And overall the best performing model was the One vs Rest. Let's try to do do some feature selection to make it even better!

Since we are using the One vs Rest algorithm backed by Logistic Regression, we can use Pyspark's Chi Squared Selector feature to select our features. I intentionally did not go over this in the previous lectures to get you used to exploring the PySpark documentation.

Here is the link to the documentation on the Chi Squared Selector for more details if you need it: https://spark.apache.org/docs/latest/ml-features#chisqselector

In [12]:
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

classifiers = [OneVsRest()]

# Select the top n features and view results
maximum = len(input_columns)
for n in range(10, maximum, 10):
    print("Testing top n = ", n, " features")

    # For Tree classifiers
    #     best_n_features = RF_featureimportances.argsort()[-n:][::-1]
    #     best_n_features= best_n_features.tolist() # convert to a list
    #     vs = VectorSlicer(inputCol="features", outputCol="best_features", indices=best_n_features)
    #     bestFeaturesDf = vs.transform(test2_data)

    # For Logistic regression or One vs Rest
    selector = ChiSqSelector(
        numTopFeatures=n,
        featuresCol="features",
        outputCol="selectedFeatures",
        labelCol="label",
    )
    bestFeaturesDf = selector.fit(test2_data).transform(test2_data)
    bestFeaturesDf = bestFeaturesDf.select("label", "selectedFeatures")
    bestFeaturesDf = bestFeaturesDf.withColumnRenamed("selectedFeatures", "features")

    # Collect features
    features = bestFeaturesDf.select(["features"]).collect()

    # Split
    train, test = bestFeaturesDf.randomSplit([0.7, 0.3])

    # Specify folds
    folds = 2

    # set up your results table
    columns = ["Classifier", "Result"]
    vals = [("Place Holder", "N/A")]
    results = spark.createDataFrame(vals, columns)

    for classifier in classifiers:
        new_result = ClassTrainEval(classifier, features, classes, folds, train, test)
        results = results.union(new_result)
    results = results.where("Classifier!='Place Holder'")
    results.show(100, False)

Testing top n =  10  features


 
OneVsRest
Intercept:  -0.06072629624683534
Top 20 Coefficients:
+-------------------+--------------------+
|feature            |coeff               |
+-------------------+--------------------+
|10-MFCCs2m         |2.1932935470219466  |
|8-SpectralRolloffm |0.9745383221820368  |
|2-Energym          |0.8794497186946554  |
|3-EnergyEntropym   |0.14045409424494978 |
|1-ZCRm             |-0.26309075278261207|
|6-SpectralEntropym |-1.5123977051094781 |
|7-SpectralFluxm    |-1.9256689496503463 |
|4-SpectralCentroidm|-2.6784917831715256 |
|9-MFCCs1m          |-3.8600697278143348 |
|5-SpectralSpreadm  |-5.733559691407162  |
+-------------------+--------------------+

None
Intercept:  -5.216974439795948
Top 20 Coefficients:
+-------------------+--------------------+
|feature            |coeff               |
+-------------------+--------------------+
|9-MFCCs1m          |4.479178913969014   |
|10-MFCCs2m         |2.8148072934816697  |
|6-SpectralEntropym |0.9513222773926217  |
|4-SpectralCentr

+-------------------+--------------------+
|feature            |coeff               |
+-------------------+--------------------+
|9-MFCCs1m          |4.526693281045776   |
|3-EnergyEntropym   |1.6804984113324575  |
|8-SpectralRolloffm |0.8600745310673594  |
|7-SpectralFluxm    |-0.29861417686060965|
|6-SpectralEntropym |-0.9477835821630294 |
|5-SpectralSpreadm  |-1.10343896598465   |
|1-ZCRm             |-1.214559065873365  |
|10-MFCCs2m         |-3.075155955908339  |
|4-SpectralCentroidm|-3.1943936105512356 |
|2-Energym          |-4.214943298762052  |
+-------------------+--------------------+

None
Intercept:  0.7651359455337734
Top 20 Coefficients:
+-------------------+-------------------+
|feature            |coeff              |
+-------------------+-------------------+
|7-SpectralFluxm    |1.614304440761214  |
|6-SpectralEntropym |1.5031804177049568 |
|10-MFCCs2m         |1.1528545326800383 |
|5-SpectralSpreadm  |-0.5429463353242856|
|3-EnergyEntropym   |-0.5699540205559211|
|2-E

 
OneVsRest
Intercept:  -1.0660890867326058
Top 20 Coefficients:
+-------------------+--------------------+
|feature            |coeff               |
+-------------------+--------------------+
|1-ZCRm             |2.723385586784241   |
|13-MFCCs5m         |2.157418522820124   |
|20-MFCCs12m        |1.7803972060888058  |
|17-MFCCs9m         |1.4541023223686755  |
|12-MFCCs4m         |1.2671603947429018  |
|16-MFCCs8m         |1.2540638105820376  |
|3-EnergyEntropym   |0.5630444879305101  |
|4-SpectralCentroidm|0.2083064571472569  |
|6-SpectralEntropym |-0.3426440626506752 |
|2-Energym          |-0.42574540417356704|
|5-SpectralSpreadm  |-0.7020873596650559 |
|11-MFCCs3m         |-0.7917884653641595 |
|14-MFCCs6m         |-1.3183730286717783 |
|15-MFCCs7m         |-1.8366173897124305 |
|18-MFCCs10m        |-1.920010273515     |
|10-MFCCs2m         |-2.3034279851188044 |
|7-SpectralFluxm    |-2.7857930462895992 |
|9-MFCCs1m          |-2.91096527814896   |
|8-SpectralRolloffm |-3.03777432

+-------------------+--------------------+
|feature            |coeff               |
+-------------------+--------------------+
|1-ZCRm             |3.192919966569719   |
|20-MFCCs12m        |2.284053951287407   |
|13-MFCCs5m         |2.2521256139138406  |
|17-MFCCs9m         |1.86321227456494    |
|6-SpectralEntropym |1.2079878248129845  |
|8-SpectralRolloffm |1.0448886679881821  |
|12-MFCCs4m         |0.7207089179178254  |
|5-SpectralSpreadm  |0.7122623093626551  |
|16-MFCCs8m         |0.46593290907445256 |
|14-MFCCs6m         |0.2650557409429376  |
|2-Energym          |0.028494743482236275|
|10-MFCCs2m         |-0.14696017710944503|
|18-MFCCs10m        |-0.16401243217619446|
|15-MFCCs7m         |-0.36672227074914626|
|7-SpectralFluxm    |-0.5918885885357373 |
|9-MFCCs1m          |-0.6329243379743054 |
|3-EnergyEntropym   |-1.0371738149698724 |
|11-MFCCs3m         |-1.3541328296528055 |
|4-SpectralCentroidm|-1.7484260629088837 |
|19-MFCCs11m        |-3.630943440048664  |
+----------

...

+---------------------+------------------+
|feature              |coeff             |
+---------------------+------------------+
|33-ChromaVector12m   |4.66804703850487  |
|35-ZCRstd            |3.3777718616896317|
|4-SpectralCentroidm  |3.0340351596437305|
|60-ChromaVector5std  |1.9318985867641036|
|32-ChromaVector11m   |1.9224620385439408|
|24-ChromaVector3m    |1.8588247686173995|
|16-MFCCs8m           |1.8521613417035263|
|37-EnergyEntropystd  |1.8274525330163265|
|30-ChromaVector9m    |1.5627051299238315|
|17-MFCCs9m           |1.3522830816938354|
|27-ChromaVector6m    |1.2859225005793604|
|20-MFCCs12m          |1.2201350269015574|
|31-ChromaVector10m   |1.210650418077261 |
|26-ChromaVector5m    |1.1321165885363065|
|6-SpectralEntropym   |1.0786765317338076|
|39-SpectralSpreadstd |0.9766497429626484|
|28-ChromaVector7m    |0.8696916199126253|
|9-MFCCs1m            |0.8200597955967599|
|18-MFCCs10m          |0.7129846525726201|
|40-SpectralEntropystd|0.619250553350447 |
+----------

+----------+------+
|Classifier|Result|
+----------+------+
|OneVsRest |45.01 |
+----------+------+

Testing top n =  70  features


 
OneVsRest
Intercept:  -5.314951653065414
Top 20 Coefficients:
+-------------------+------------------+
|feature            |coeff             |
+-------------------+------------------+
|9-MFCCs1m          |3.699724770701113 |
|5-SpectralSpreadm  |1.9123157209922095|
|43-MFCCs1std       |1.8813402875458514|
|52-MFCCs10std      |1.606727425493536 |
|70-BPMconf         |1.5898401156350925|
|12-MFCCs4m         |1.5800846164844944|
|15-MFCCs7m         |1.3109537179592663|
|54-MFCCs12std      |1.285688879130781 |
|24-ChromaVector3m  |1.2616868986182255|
|64-ChromaVector9std|1.1660219939665397|
|20-MFCCs12m        |1.1463276159000464|
|11-MFCCs3m         |1.1040797033336576|
|53-MFCCs11std      |1.0659005404270077|
|51-MFCCs9std       |1.032999872625775 |
|22-ChromaVector1m  |1.0212860471691303|
|2-Energym          |0.9591839832714432|
|47-MFCCs5std       |0.9316637474738771|
|30-ChromaVector9m  |0.9119487765503973|
|58-ChromaVector3std|0.8934613210459827|
|3-EnergyEntropym   |0.85749669235

+---------------------+------------------+
|feature              |coeff             |
+---------------------+------------------+
|11-MFCCs3m           |2.358587280314608 |
|55-MFCCs13std        |2.1191943557837707|
|64-ChromaVector9std  |2.0958668903005333|
|62-ChromaVector7std  |1.9677553705891753|
|40-SpectralEntropystd|1.9437795544123921|
|65-ChromaVector10std |1.8845673189603054|
|17-MFCCs9m           |1.8494846640293319|
|63-ChromaVector8std  |1.8394102438194768|
|67-ChromaVector12std |1.6517687095947726|
|12-MFCCs4m           |1.6008558803321584|
|19-MFCCs11m          |1.575774613213961 |
|10-MFCCs2m           |1.4923586489470582|
|43-MFCCs1std         |1.456207634971904 |
|56-ChromaVector1std  |1.3773852231942885|
|57-ChromaVector2std  |1.2740786126641817|
|16-MFCCs8m           |1.26200081694397  |
|60-ChromaVector5std  |1.2471864717630312|
|22-ChromaVector1m    |1.2348562764764073|
|68-ChromaDeviationstd|1.1925243220575878|
|48-MFCCs6std         |1.1463446021078327|
+----------

+----------------------+------------------+
|feature               |coeff             |
+----------------------+------------------+
|43-MFCCs1std          |3.5565658819343318|
|41-SpectralFluxstd    |3.2655827859954716|
|7-SpectralFluxm       |2.970135004838184 |
|21-MFCCs13m           |1.8085875119809651|
|45-MFCCs3std          |1.703451346487819 |
|39-SpectralSpreadstd  |1.6042049784462846|
|38-SpectralCentroidstd|1.51317620381767  |
|29-ChromaVector8m     |1.4815941875222849|
|34-ChromaDeviationm   |1.428772741166039 |
|37-EnergyEntropystd   |1.4248106411764183|
|47-MFCCs5std          |1.3104295218576445|
|20-MFCCs12m           |1.2724064070608987|
|33-ChromaVector12m    |1.2217914997877892|
|27-ChromaVector6m     |1.208870453379927 |
|70-BPMconf            |1.113002446873609 |
|10-MFCCs2m            |0.9992710849828313|
|25-ChromaVector4m     |0.9816168130780197|
|23-ChromaVector2m     |0.944888438999317 |
|28-ChromaVector7m     |0.9379513210313015|
|58-ChromaVector3std   |0.879522

# Results

Let's see if we can improve upon the results above of 45.94%.

- 10 features = 32.27% (going down)
- 20 features = 39.23% (going down)
- 30 features = 39.24% (going down)
- 40 features = 42.61% (going down)
- 50 features = 43.52% (going down)
- 60 features = 42.42% (going down)
- 70 features = 40.22% (going down)

**Take away:** looks like we actually never beat our baseline so let's stick with the 71 features. Setting a random seed might have helped with variability we saw here.

## Train final model

Now we can finally train our final model with the optimal set of features!

In [13]:
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

classifiers = [OneVsRest()]

# Select the top n features and view results
n = 71

# For Logistic regression or One vs Rest
selector = ChiSqSelector(
    numTopFeatures=n,
    featuresCol="features",
    outputCol="selectedFeatures",
    labelCol="label",
)
bestFeaturesDf = selector.fit(test2_data).transform(test2_data)
bestFeaturesDf = bestFeaturesDf.select("label", "selectedFeatures")
bestFeaturesDf = bestFeaturesDf.withColumnRenamed("selectedFeatures", "features")

# Collect features
features = bestFeaturesDf.select(["features"]).collect()

# Split
train, test = bestFeaturesDf.randomSplit([0.7, 0.3])

# Specify folds
folds = 2

# set up your results table
columns = ["Classifier", "Result"]
vals = [("Place Holder", "N/A")]
results = spark.createDataFrame(vals, columns)

for classifier in classifiers:
    new_result = ClassTrainEval(classifier, features, classes, folds, train, test)
    results = results.union(new_result)
results = results.where("Classifier!='Place Holder'")
results.show(100, False)

 
OneVsRest
Intercept:  -4.0993983607167666
Top 20 Coefficients:
+---------------------+------------------+
|feature              |coeff             |
+---------------------+------------------+
|9-MFCCs1m            |4.046041594813302 |
|71-BPMessentia       |2.4557843319181534|
|5-SpectralSpreadm    |1.9046320025029742|
|24-ChromaVector3m    |1.6476956156040723|
|52-MFCCs10std        |1.5963617173425653|
|22-ChromaVector1m    |1.4786560268998832|
|63-ChromaVector8std  |1.2018632612565407|
|50-MFCCs8std         |1.0733443946802659|
|12-MFCCs4m           |0.9120498377293653|
|3-EnergyEntropym     |0.8793237223196377|
|2-Energym            |0.8726271275644704|
|54-MFCCs12std        |0.8434855745015865|
|62-ChromaVector7std  |0.727146714644866 |
|36-Energystd         |0.7094159679871512|
|53-MFCCs11std        |0.69641989426862  |
|20-MFCCs12m          |0.6874453628782129|
|18-MFCCs10m          |0.6623171116448462|
|47-MFCCs5std         |0.5460199359115526|
|64-ChromaVector9std  |0.5328920

+---------------------+------------------+
|feature              |coeff             |
+---------------------+------------------+
|63-ChromaVector8std  |3.4742632257347927|
|17-MFCCs9m           |2.2792793589330462|
|11-MFCCs3m           |2.2196767181061645|
|62-ChromaVector7std  |2.1295477756027577|
|40-SpectralEntropystd|2.0453554173257857|
|55-MFCCs13std        |1.9106548048182195|
|68-ChromaDeviationstd|1.7872219115648298|
|19-MFCCs11m          |1.7221763148303828|
|51-MFCCs9std         |1.597835039460991 |
|46-MFCCs4std         |1.4900566056904165|
|48-MFCCs6std         |1.472058241751189 |
|32-ChromaVector11m   |1.3842596779251832|
|64-ChromaVector9std  |1.369976522908649 |
|10-MFCCs2m           |1.3647944632167046|
|12-MFCCs4m           |1.3385189724191389|
|60-ChromaVector5std  |1.2561798042916277|
|67-ChromaVector12std |1.203100284303376 |
|43-MFCCs1std         |1.1251178829305963|
|22-ChromaVector1m    |1.036439245974023 |
|57-ChromaVector2std  |0.9591105229540422|
+----------

+----------------------+------------------+
|feature               |coeff             |
+----------------------+------------------+
|41-SpectralFluxstd    |3.6592367434347874|
|7-SpectralFluxm       |3.4295011086516087|
|29-ChromaVector8m     |3.105983088742999 |
|43-MFCCs1std          |2.8088856633862873|
|39-SpectralSpreadstd  |2.133720412989536 |
|38-SpectralCentroidstd|2.0898058145399827|
|21-MFCCs13m           |1.959072236674976 |
|27-ChromaVector6m     |1.6934030996701674|
|33-ChromaVector12m    |1.6857451484995918|
|45-MFCCs3std          |1.5151611347303537|
|71-BPMessentia        |1.4800420493380761|
|10-MFCCs2m            |1.3557106468573745|
|30-ChromaVector9m     |1.275748964875869 |
|47-MFCCs5std          |1.2522442525646833|
|37-EnergyEntropystd   |1.0980552047916463|
|20-MFCCs12m           |1.0609389507154354|
|13-MFCCs5m            |1.0568843584101397|
|25-ChromaVector4m     |0.9983238349110527|
|59-ChromaVector4std   |0.9177386888219036|
|28-ChromaVector7m     |0.820179

+----------+------+
|Classifier|Result|
+----------+------+
|OneVsRest |48.35 |
+----------+------+



## Make a recommendation to a user!

Imagine that your new model is being deployed in the next version release of the online radio station that has now purchased this model from you. So the existing users have gotten used to the songs in certian stations but now your model may be adding new songs to those stations. 

Can you show an output that lists songs that will be new to listers of the BigRoom station (ie. songs that used to belong to another station, but are now classified by your model as being in the BigRoom station)? Also show how many songs this would include. You can use the test dataframe to do this. Don't over think it. 

In [14]:
predictions = OVR_BestModel.transform(test)

In [15]:
# From the output earlier we saw that the new label for BigRoom is now 21.0
# Let's get a song from there
count = predictions.filter("label!=21.0 AND prediction == 21.0").count()
print(count)
predictions.filter("label!=21.0 AND prediction == 21.0").show()
# predictions.show()

14


+-----+--------------------+--------------------+----------+
|label|            features|       rawPrediction|prediction|
+-----+--------------------+--------------------+----------+
|  3.0|[0.06536850149790...|[-5.8798110067602...|      21.0|
| 12.0|[0.23144431675954...|[-4.5987443858441...|      21.0|
| 12.0|[0.50938702508293...|[-7.0472836215512...|      21.0|
| 16.0|[0.26174259188837...|[-6.6792988974869...|      21.0|
| 16.0|[0.34345894073264...|[-5.1080635187261...|      21.0|
| 17.0|[0.12144798835851...|[-6.5793229917827...|      21.0|
| 17.0|[0.21246687650121...|[-4.7579631079554...|      21.0|
| 17.0|[0.24690950657943...|[-6.2469209832735...|      21.0|
| 17.0|[0.45459472182908...|[-7.1577398098736...|      21.0|
| 18.0|[0.32659039393572...|[-5.4825148143828...|      21.0|
| 19.0|[0.25509374714148...|[-8.6992293525434...|      21.0|
| 20.0|[0.21963633400178...|[-8.5315842342282...|      21.0|
| 20.0|[0.26427395761486...|[-5.9129594803795...|      21.0|
| 20.0|[0.34565765181533

## Conclusions

#### Other types of analyses to consider

1. **Training and evaluation splits::** we did a 70/30 split but it's a good idea to play around with other split ratios like 80/20 or 75/25
2. **Different transformations:** you could also play around with other types of tranformations on the data like normalizing or standard scaling:
        - normalizing: https://spark.apache.org/docs/2.1.0/ml-features.html#normalizer
        - standard scaling: https://spark.apache.org/docs/2.1.0/ml-features.html#standardscaler